# ハフマン符号

### huffman 関数

1. 文字列`string`を受け取る
2. 含まれる文字をカウントして出現頻度を`print`する
3. ハフマン符号用の木にする
4. 木を解析して、それぞれの文字を符号にして`print`する
5. 文字列(`string`)をエンコードして返す

### 追加機能

この機能については **特に** 動作の保証はありません

1. 文字列`string`、割り当てられた文字と割り当てられた符号のタプル`assign`を受け取る
2. 含まれる文字をカウントして出現頻度を`print`する
3. ハフマン符号用の木にする
4. 木を解析して、それぞれの文字を符号にする
5. 割り当てられた文字と割り当てられた符号のタプル`assign`をもとに符号を改変して`print`する
6. 文字列(`string`)をエンコードして返す


In [1]:
from fractions import Fraction
import heapq
from typing import Optional


class Node:
    def __init__(self, character: str, probability: float, left, right) -> None:
        self.character = character
        self.probability = probability
        self.left: Node | None = left
        self.right: Node | None = right

    def __lt__(self, other):
        return self.probability < other.probability


def to_bit(node: Node, bit: str, bits: dict[str, str]):
    if node.left is None and node.right is None:
        bits[node.character] = bit
    if node.left is not None:
        to_bit(node.left, bit + "1", bits)
    if node.right is not None:
        to_bit(node.right, bit + "0", bits)


def huffman(string: str, assign: Optional[tuple[str, str]] = None):
    dictionary: dict[str, int] = {}
    total_chars = len(string)
    for s in string:
        if s in dictionary:
            dictionary[s] += 1
        else:
            dictionary[s] = 1
    print("各文字の出現頻度:", dictionary)
    print("各文字の出現確率:")
    for char, freq in dictionary.items():
        print(f"'{char}': {Fraction(freq, total_chars)}")
    nodes: list[Node] = []
    heapq.heapify(nodes)
    for s in dictionary:
        heapq.heappush(nodes, Node(s, dictionary[s], None, None))
    while len(nodes) > 1:
        left = heapq.heappop(nodes)
        right = heapq.heappop(nodes)
        new_node = Node(
            left.character + right.character,
            left.probability + right.probability,
            left,
            right,
        )
        heapq.heappush(nodes, new_node)
    top = heapq.heappop(nodes)
    bits: dict[str, str] = {}
    to_bit(top, "", bits)
    if assign != None:
        tmp = str(int(bits[assign[0]]) ^ int(assign[1]))
        for char, freq in bits.items():
            bits[char] = (
                "0" * (len(freq) - 1)
                + bin(int(freq, 2) ^ (int(tmp, 2) >> (len(tmp) - len(freq))))[2:]
            )
        print("文字のビット表現:", bits)
    else:
        print("文字のビット表現:", bits)
    result: str = ""
    for s in string:
        result += bits[s]
    return result

### 使用例


In [2]:
string = "AAAAAAAAAAAAAAAACCGT"

print(huffman(string))
print(len(huffman(string, ("T", "000"))) / (len(string) * 2))

各文字の出現頻度: {'A': 16, 'C': 2, 'G': 1, 'T': 1}
各文字の出現確率:
'A': 4/5
'C': 1/10
'G': 1/20
'T': 1/20
文字のビット表現: {'C': '11', 'G': '101', 'T': '100', 'A': '0'}
00000000000000001111101100
各文字の出現頻度: {'A': 16, 'C': 2, 'G': 1, 'T': 1}
各文字の出現確率:
'A': 4/5
'C': 1/10
'G': 1/20
'T': 1/20
文字のビット表現: {'C': '01', 'G': '001', 'T': '000', 'A': '1'}
0.65
